In [32]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%cd '/content/drive/My Drive/'

Mounted at /content/drive
/content/drive/My Drive


In [2]:
!pip install leidenalg igraph

In [33]:
import pandas as pd
import scipy.io as sio
import numpy as np
import igraph as ig
import leidenalg
from itertools import combinations
mat=sio.loadmat('A.mat')['A'] #Import the multiplex network of sparse kernel matrices
mat

array([[[0.99999934, 1.        , 1.        ],
        [0.99998087, 0.99997668, 0.99997309],
        [0.99968988, 0.99998698, 0.99971787],
        ...,
        [0.99913058, 0.99961993, 0.99917788],
        [0.9992869 , 0.99999078, 0.99932968],
        [0.99885736, 0.99992227, 0.99891167]],

       [[0.99996816, 0.99997668, 0.99997309],
        [0.99999979, 1.        , 1.        ],
        [0.99949654, 0.99999851, 0.99951677],
        ...,
        [0.99882273, 0.99940841, 0.99885376],
        [0.99900587, 0.99993813, 0.99903437],
        [0.99850793, 0.99981381, 0.99854289]],

       [[0.99998044, 0.99998698, 0.99971787],
        [0.99999941, 0.99999851, 0.99951677],
        [0.9995498 , 1.        , 1.        ],
        ...,
        [0.99890493, 0.99946627, 0.99985886],
        [0.99908129, 0.99995584, 0.99991725],
        [0.99860064, 0.99984563, 0.99973757]],

       ...,

       [[0.99965099, 0.99961993, 0.99917788],
        [0.99943036, 0.99940841, 0.99885376],
        [0.99999645, 0

In [16]:
def Divisive_Clustering(X):
    """
    Divisive Kernel Clustering using the Leiden algorithm.

    Arguments:
    X : 3D numpy array of shape (N, N, L), where N is the number of nodes,
        and L is the number of layers (each representing a weighted adjacency matrix).

    Returns:
    Opt_rank : The optimal number of clusters to extract
    M : Community assignment of nodes
    Q : Modularity value of the partition
    aggregated_vs : Co-membership matrix aggregated across layers
    """
    # Validate input dimensions
    if len(X.shape) != 3 or X.shape[0] != X.shape[1]:
        raise ValueError("Input X must be a 3D numpy array with shape (N, N, L).")

    N, _, L = X.shape  # Number of nodes and layers
    Vs = []  # To store co-membership matrices from each layer
    combos_s = list(combinations(range(N), 2))  # Generate pairs of nodes

    for i in range(L):
        v = []  # Co-membership vector for the current layer
        net = X[:, :, i]  # Extract the i-th layer as a NumPy array

        # Create an igraph directed graph for each layer
        G = ig.Graph.Weighted_Adjacency(net.tolist(), mode="directed", attr="weight")
        partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights="weight")

        M = partition.membership

        # Compute co-membership matrix for the current layer
        co_matrix = np.zeros((N, N))
        for ii in range(len(combos_s)):
            m1 = M[combos_s[ii][0]]
            m2 = M[combos_s[ii][1]]
            if m1 == m2:
                co_matrix[combos_s[ii][0], combos_s[ii][1]] = net[combos_s[ii][0], combos_s[ii][1]]+net[combos_s[ii][1], combos_s[ii][0]]
                co_matrix[combos_s[ii][1], combos_s[ii][0]] = net[combos_s[ii][0], combos_s[ii][1]]+net[combos_s[ii][1], combos_s[ii][0]]  # Ensure symmetry

        Vs.append(co_matrix)

    # Convert the list of co-membership matrices to a 3D array
    Vs = np.stack(Vs, axis=2)

    # Aggregate the co-membership matrices by summing across layers
    aggregated_vs = np.sum(Vs, axis=2)

    # Create a graph for the aggregated matrix (using undirected mode)
    # Aggregated co-membership matrix should be treated as undirected
    G = ig.Graph.Weighted_Adjacency(aggregated_vs.tolist(), mode="undirected", attr="weight")
    partition = leidenalg.find_partition(G, leidenalg.ModularityVertexPartition, weights="weight")

    Q = partition.q  # Modularity score
    M = partition.membership

    # Return the optimal number of clusters and other results
    Opt_rank = max(M) + 1
    return Opt_rank, M, aggregated_vs



In [34]:
Opt_rank, M, aggregated_vs= Divisive_Clustering(mat)
print("Optimal number of clusters:", Opt_rank)
print("Community assignments:", M)
print("Aggregated co-membership matrix (Vs):\n", aggregated_vs)


Optimal number of clusters: 2
Community assignments: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1]
Aggregated co-membership matrix (Vs):
 [[0.         5.99984858 5.99908001 ... 3.99802145 3.99926351 3.99863786]
 [5.99984858 0.         5.99852651 ... 3.99706991 3.9988272  3.99796159]
 [5.99908001 5.99852651 0.         ... 3.99783391 3.99878056 3.99821454]
 ...
 [3.99802145 3.99706991 3.99783391 ... 0.         5.99869106 5.99910223]
 [3.99926351 3.9988272  3.99878056 ... 5.99869106 0.         5.9985655 ]
 [3.99863786 3.99796159 3.99821454 ... 5.99910223 5.9985655  0.        ]]


In [9]:
def recursive_Divisive_Clustering(X, max_depth=3, current_depth=0):
    """
    Divisive Kernel Clustering with recursion using the Leiden algorithm.

    Arguments:
    X : 3D numpy array representing the graph layers.
    max_depth : Maximum recursion depth.
    current_depth : Current recursion depth (used for tracking).

    Returns:
    tree_structure : A tree-like dictionary of the community detection results or None if stopped.
    """
    # Stop recursion if the current depth exceeds the max_depth
    if current_depth >= max_depth:
        return None

    # Apply the consensus function to get results
    Opt_rank, M, aggregated_vs = Divisive_Clustering(X)

    # If Opt_rank is 1, stop recursion and return None (this branch will not have any output)
    if Opt_rank == 1:
        return None

    # Prepare the tree structure at the current depth
    result_tree = {
        'Depth': current_depth,
        'Community Membership': M,
        'Aggregated Co-membership Matrix': aggregated_vs,
    }

    # Create subgraphs based on community membership
    subgraphs = {}
    N = len(M)
    for community_id in set(M):
        # Create a subgraph for each community
        community_nodes = [i for i in range(N) if M[i] == community_id]
        if len(community_nodes) > 1:  # Only recurse on non-trivial subgraphs
            subgraph_data = X[community_nodes, :][:, community_nodes, :]
            subgraphs[community_id] = recursive_Divisive_Clustering(subgraph_data, max_depth, current_depth + 1)

    # Only add subgraphs if there are any, otherwise return None
    if subgraphs:
        result_tree['Subgraphs'] = subgraphs
    else:
        result_tree['Subgraphs'] = {}

    return result_tree



In [35]:
# Example usage
# Assuming `mat` is your 3D numpy array (shape: N x N x L)
result_tree = recursive_Divisive_Clustering(mat, max_depth=8,current_depth=0)
result_tree

{'Depth': 0,
 'Community Membership': [0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  1,
  1,
  1],
 'Aggregated Co-membership Matrix': array([[0.        , 5.99984858, 5.99908001, ..., 3.99802145, 3.99926351,
         3.99863786],
        [5.99984858, 0.        , 5.99852651, ..., 3.99706991, 3.9988272 ,
         3.99796159],
        [5.99908001, 5.99852651, 0.        , ..., 3.99783391, 3.99878056,
         3.99821454],
        ...,
        [3.99802145, 3.99706991, 3.99783391, ..., 0.        , 5.99